In [1]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [3]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [4]:
# !pip install aif360

In [5]:
# !pip install BlackBoxAuditing==0.1.54

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [8]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/dmytroomelian/PycharmProjects/Experiments-AAAI-24


## Import dependencies

In [9]:
import os
import copy
import numpy as np

from virny.utils.custom_initializers import create_config_obj

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

In [10]:
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

In [11]:
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [12]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_student_performance'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = np.linspace(0, 50, 11)
# FAIR_INTERVENTION_PARAMS_LST = [0.0]
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
# FAIR_INTERVENTION_PARAMS_LST = [0.6, 0.7, 0.8, 0.9, 1.0]
# FAIR_INTERVENTION_PARAMS_LST = [0.1]
# FAIR_INTERVENTION_PARAMS_LST = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# FAIR_INTERVENTION_PARAMS_LST = [1, 5, 10, 25, 50]
#
config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'student_performance_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [13]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

In [16]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [17]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    # 'session_uuid': '6f0b4970-ea21-4e40-a946-5fe50a4a3565', (LAST)
    # 'session_uuid': '91249614-e7df-4b80-8c4e-110ef587b414',
    # 'session_uuid': 'a10b8c6b-a184-4e02-a873-8240f5c798ed', # lfr test #1 positive
    # 'session_uuid': '27fd2ec6-5813-4516-a411-992936b50ea1',
    # 'session_uuid': '090abd0e-ab4d-4db4-a97b-60ed17c0a814', # test same uuid to see line bands
    # 'session_uuid': '4cfe1f1a-acee-4f52-987f-9fa64345ca4d',
    # 'session_uuid': 'ef8d0900-9015-42bc-a3d1-f3ae02155b5f',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  ee771c1d-f6ff-4267-a351-7d02ea98b5bb


## Initialize custom objects

In [18]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [19]:
from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset

data_loader = StudentPerformanceDataset(Config.file_name)
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,other,other,home,father,F


In [20]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [21]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True,
                                   dataset_name='StudentPerformance')

2023-11-21 15:39:18 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[ 0.  5. 10. 15. 20. 25. 30. 35. 40. 45. '
                                 '50.]',
 'model_init_seed': 200,
 'session_uuid': 'ee771c1d-f6ff-4267-a351-7d02ea98b5bb'}


2023-11-21 15:39:18 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([268, 360,  50, 378,  92, 365, 217, 122, 213,  56, 211, 235,  16,
            166, 371,  49, 250, 315, 265, 150],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([268, 360,  50, 378,  92, 365, 217, 122, 213,  56, 211, 235,  16,
            166, 371,  49, 250, 315, 265, 150],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

intervention_param:  0.0


KeyError: "None of [Index(['RACE'], dtype='object')] are in the [columns]"